# Introduction

The notebook is intended to perform a Sales Forecast over the data of the [Tabular Playground Series - September 2022](https://www.kaggle.com/competitions/tabular-playground-series-sep-2022)

In [ ]:
# Import Standard Libraries
import pandas as pd
import numpy as np
from itertools import product as cartesian_product

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedShuffleSplit

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'figure.figsize': (16, 6),
    'font.family': 'Andale Mono',
    'axes.titlesize': 24,
    'figure.facecolor': '#E5E8E8',
    'axes.facecolor': '#E5E8E8'
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('deep'), 
              rc=theme_parameters)

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv('./../../data/2022_09/train.csv',
                         parse_dates=['date'],
                         index_col=0)
test_data = pd.read_csv('./../../data/2022_09/test.csv',
                        parse_dates=['date'],
                        index_col=0)

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Exploratory Data Analysis

## Null Values

In [ ]:
# Define data to display
data_to_display = {
    'Train Data': train_data,
    'Test Data': test_data
}

# Create figure
figure = plt.figure(tight_layout=True, figsize=(16, 6))


# Define figure's subplots
grid_spec = GridSpec(len(data_to_display), 1)

# Fetch data to plot
for row, (name, data) in enumerate(data_to_display.items()):
    
    # Create subplot
    ax = figure.add_subplot(grid_spec[row, 0])
    
    sns.histplot(data=data.isna().melt(var_name='Feature', value_name='Missing'), 
                 y='Feature', 
                 hue='Missing', 
                 multiple='fill',
                 ax=ax)
    
    # Refactor x label as missing percentage
    ax.xaxis.set_major_locator(ticker.LinearLocator(21))
    ax.xaxis.set_major_formatter('{:.0%}'.format)
    
    # Set title and label
    ax.set_title(name, fontsize=15)
    ax.set_xlabel('Null Percentage', fontweight='bold')

There are no null values, so there is not need to impute the data while training or testing the model.

## Distribution of Sold Product per Country, Store and Product

In [ ]:
# Define the subplot
figure, ax = plt.subplots(12, 4, figsize=(25, 50))
ax = ax.flatten()

# Fetch the data to plot
for index, (combination, data_plot) in enumerate(train_data.groupby(['country', 'store', 'product'])):
    
    sns.histplot(data=data_plot.num_sold, 
                 ax=ax[index])
    
    ax[index].set_title(' | '.join(combination), fontsize=14)

plt.tight_layout()

Most of the countries have a skewed distribution, except Poland. 

## Sold Product per Country

In [ ]:
# Plot the number of sold products per country with respect to product
ax = sns.barplot(data=train_data, 
                 x='product', 
                 y='num_sold', 
                 hue='country')

ax.set_title('Product Sold per Country')

plt.show()

- Every product is sold by every country
- Kaggle for Kids looks like the most sold

## Sold Product per Country over Time

In [ ]:
# Plot the number of sold products per country with respect to time
ax = sns.lineplot(
    data=train_data.groupby([
        train_data.date.dt.strftime('%Y-%m'), train_data.country 
    ])['num_sold'].sum().reset_index(), 
    x='date', 
    y='num_sold', 
    hue='country'
)

ax.set_title('Number of Sold Products per Country')

plt.xticks(rotation=45)

ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=train_data.date.dt.strftime('%Y-%m').nunique()/2))

plt.show()

- There is a non-stationary point around early 2020

## Sold Product per Product over Time

In [ ]:
# Define subplot
figure, ax = plt.subplots(2, 2, figsize=(16, 9))
ax = ax.flatten()
hue_order = train_data['country'].unique()

# Fetch the products
for index, product in enumerate(train_data['product'].unique()):
    
    # Retrieve data
    data_plot = train_data[train_data['product'] == product]
        
    # Plot
    sns.lineplot(data=data_plot.groupby([
                        data_plot.date.dt.strftime('%Y-%m'), 
                        data_plot.country
                        ])['num_sold'].sum().reset_index(), 
                 x='date', 
                 y='num_sold', 
                 hue='country',
                 hue_order=hue_order,
                 ax=ax[index])
    
    # Retrieve legends information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()
    
    # Set the title
    ax[index].set_title(product, fontsize=14)
    ax[index].xaxis.set_major_locator(ticker.MaxNLocator(nbins=data_plot.date.dt.strftime('%Y-%m').nunique()/5))
    
# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=train_data['country'].nunique())
    
plt.tight_layout()

- As per the previous plot, there is a strange behavior in 2020
- `Kaggle Advanced Techniques` and `Kaggle Getting Started` have a month seasonality. It could be useful to use the month as a feature.

## Sold Product per Product & Store over Time

In [ ]:
# Define the subplot
figure, ax = plt.subplots(4, 2, figsize=(16, 12), sharey=True)
ax = ax.flatten()
hue_order = train_data['country'].unique()

# Compute every possible combination between 'product' and 'store'
products = train_data['product'].unique()
stores = train_data['store'].unique()
iterator = cartesian_product(products, stores)

# Fetch the data to plot
for index, (product, store) in enumerate(iterator):
    
    # Retrieve the data to plot
    data_plot = train_data[
        (train_data['store'] == store) &
        (train_data['product'] == product)]
    
    # Plot the data
    sns.lineplot(
        data=data_plot.groupby([
            data_plot.date.dt.strftime('%Y-%m'), 
            data_plot.country
        ])['num_sold'].sum().reset_index(),
        x='date',
        y='num_sold',
        hue='country',
        hue_order=hue_order,
        ax=ax[index])
    
    # Set title
    ax[index].set_title(f'{store} | {product}', 
                        fontsize=10)
    
    # Set bins
    ax[index].xaxis.set_major_locator(ticker.MaxNLocator(nbins=data_plot.date.dt.strftime('%Y-%m').nunique()/5))
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()
    

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=train_data['country'].nunique())
    
plt.tight_layout()                 

- `KaggleRama` clarly sells less than `KaggleMart`. The store has to be a feature.
- Non-stationary point in 2020
- Monthly seasonality for `Kaggle Advanced Techniques` and `Kaggle Getting Started`

## Seasonality

In [ ]:
def plot_seasonality(seasonality_column, ci=95, alpha=1):
    """
    Utility function for plot the specified seasonality in 'seasonality_column' with respect to the 'product' and 'store' grouping
    
    Parameters:
        seasonality_column:
        ci: Integer confidence interval value
        alpha
    """
    
    # Define the subplots
    figure, ax = plt.subplots(4, 2, figsize=(18, 20))
    ax = ax.flatten()
    hue_order = train_data['country'].unique()
    
    
    # Compute every possible combination between 'product' and 'store'
    products = train_data['product'].unique()
    stores = train_data['store'].unique()
    iterator = cartesian_product(products, stores)
    
    # Fetch the data to plot
    for index, (product, store) in enumerate(iterator):
        
        # Retrieve the data to plot
        data_plot = train_data[
            (train_data['store'] == store) &
            (train_data['product'] == product)]
    
        # Plot the data
        sns.lineplot(
            data=pd.melt(
                data_plot, 
                id_vars=['country', seasonality_column],
                value_vars=['num_sold'],
                value_name='num_solds'
            ),
            x=seasonality_column,
            y='num_solds',
            hue='country',
            ax=ax[index],
            errorbar=('ci', ci),
            alpha=alpha,
            hue_order=hue_order
        )
        
        # Set Title
        ax[index].set_title(f'{store} | {product}', 
                             fontsize=12)
        
        # Retrieve legend information
        handles = ax[index].get_legend_handles_labels()[0]
        labels = ax[index].get_legend_handles_labels()[1]
        ax[index].legend().remove()

    # Set the legend
    figure.legend(handles, 
                  labels, 
                  loc='upper center', 
                  bbox_to_anchor=(0.5, 1.03), 
                  fontsize=12,
                  ncol=train_data['country'].nunique())
    
    # Set the figure title
    plt.suptitle(f'Seasonality by {seasonality_column}',
                 fontsize=16)
    
    plt.tight_layout()
    
    plt.show()

### Engineering Date

In [ ]:
def engineer_date(data: pd.DataFrame):
    """
    Engineer the 'date' column to retrieve meaningful information such as the day, month, year, day of week, quarter, day of year and week of year
    
    Parameters:
        data: Pandas DataFrame with the 'date' column
    
    Return:
        data: Pandas DataFrame with the engineered date columns
    """
    
    # Day
    data['day'] = data['date'].dt.day
    
    # Month
    data['month'] = data['date'].dt.month
    
    # Year
    data['year'] = data['date'].dt.year
    
    # Day of the week
    data['dayofweek'] = data['date'].dt.dayofweek
    
    # Quarter
    data['quarter'] = data['date'].dt.quarter
    
    # Day of the year
    data['dayofyear'] = data['date'].dt.dayofyear
    
    # Week of the year
    data['weekofyear'] = data['date'].dt.isocalendar().week
    data.loc[data['weekofyear'] > 52, 'weekofyear'] = 52 # Not every year has 53 weeks, so let's cap to 52
    data['weekofyear'] = data['weekofyear'].astype('int64') # Convert back the column to integer

In [ ]:
# apply the 'engineer_date' function over the training and test sets
engineer_date(train_data)
engineer_date(test_data)

### Day

In [ ]:
plot_seasonality('day')

- There is a slightly increase toward the end of the month. Create a boolean variable for the last 27-30 days.

### Month

In [ ]:
plot_seasonality('month')

- There is an increasing trend towards the end of the year
- `Kaggle Advanced Techniques` is more popular at the start of the year, while `Kaggle Getting Started`is more popular at the end

### Year

In [ ]:
plot_seasonality('year')

- Quite a strange behavior in 2020, like a convergence point

### Day of Week

In [ ]:
plot_seasonality('dayofweek')

- Increasing trend over the Weekend. It could be usefull to encode it as a feature.

### Quarter

In [ ]:
plot_seasonality('quarter')

- Same seasonality as Month

### Day of Year

In [ ]:
plot_seasonality('dayofyear')

### Week of the Year

In [ ]:
plot_seasonality('weekofyear')

- Seasonality for the same product is the same across the two different stores
- `Kaggle Advanced Techniques` is more popular at the start of the year, while `Kaggle Getting Started`is more popular at the end
- Spike at the end of the year across all products and all stores

# Data Preparation

## Product Normalization

In [ ]:
# Replace spaces and special characters
train_data['product'] = train_data['product'].str.replace(' ', '_')
train_data['product'] = train_data['product'].str.replace(':', '_')
test_data['product'] = test_data['product'].str.replace(' ', '_')
test_data['product'] = test_data['product'].str.replace(':', '_')

## Features and Labels Definition

In [ ]:
# Define features and labels
numerical_features = ['date_day', 
                      'date_month', 
                      'date_year', 
                      'date_dayofweek']

categorical_features = ['country', 
                        'store', 
                        'product']

labels = ['num_sold']

## Numerical Features

Since there are no NaN values in the data, it is possible to skip the Fill NaN values step.

In [ ]:
# Numerical features pipeline
numerical_features_pipeline = Pipeline(steps=[
    ('numerical_scaler', StandardScaler())
])

## Categorical Features

Since there are no NaN values in the data, it is possible to skip the Fill NaN values step.

In [ ]:
# Categorical features pipeline
categorical_features_pipeline = Pipeline(steps=[
    ('categorical_one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

## Bundle Data Preprocessing Steps

In [ ]:
# Bunlde data preprocessing steps
data_preprocessor = ColumnTransformer(
    transformers=[
        ('numerical_preprocessing', numerical_features_pipeline, numerical_features),
        ('categorical_preprocessing', categorical_features_pipeline, categorical_features)
    ])

# Train & Test Split

Use the data from 2020 as validation set.

In [ ]:
# Define X and y for the training set
X = train_data[numerical_features + categorical_features]
y = train_data[labels]

In [ ]:
# Split training data into train and validation
X_train = X[X['date_year'] != 2020]
y_train = y.iloc[X_train.index]
X_test = X[X['date_year'] == 2020]
y_test = y.iloc[X_test.index]

In [ ]:
# Define the a Stratified K-fold Shuffle Splitter
stratified_kfold = StratifiedShuffleSplit(n_splits=5,
                                          test_size=.3, 
                                          random_state=0)

# Model

In [ ]:
# Define function to compute the SMAPE (source: https://www.kaggle.com/code/jcaliz/tps-sep22-eda-baseline-you-were-looking-for)
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

In [ ]:
# Define the used metrics
metrics = ['smape', 'mse']

In [ ]:
# Initialize DataFrame of model performance
performance = pd.DataFrame(columns=metrics)

## Linear Regression

In [ ]:
%%time

# Define the model
model_lr = LinearRegression()

# Define the pipeline
pipe_lr = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('linear_regression', model_lr)
])

# Train the pipeline
pipe_lr.fit(X_train, 
            y_train)

# Get number of sold predictions
predictions_lr = pipe_lr.predict(X_test)

# Model evaluation
smape_lr = round(smape(y_test, predictions_lr), 2)
mse_lr = round(mean_squared_error(y_test, predictions_lr), 2)

print('SMAPE: {}%'.format(smape_lr))
print('MSE: {}'.format(mse_lr))
print('\n')

In [ ]:
# Update 'performance' DataFrame
performance.loc['linear_regression'] = [smape_lr, mse_lr]

## Linear Regression - Cross-Validation

# Model Comparison